! pip install pdfkit
! pip install wkhtmltopdf
https://wkhtmltopdf.org/downloads.html

Code made to save the knowledge that Mr. Gaves shares with us through his newsletter, Institut Des Libertés (https://institutdeslibertes.org/qui-sommes-nous/).

In [ ]:
# 1) Cartographie les articles
from bs4 import BeautifulSoup
import requests
import time

def grab_data(page) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = "https://institutdeslibertes.org/auteur/charlesgave/page/{}/".format(page)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    return html_soup

def save_file(name, file) :
    with open("{}.json".format(name), "w") as write_file:
        json.dump(file, write_file, indent=4)
    print("Saved")

all_links = []
for i in range(1,75) :  # 74 pages
    print(i)
    html_soup = grab_data(i)
    articles = html_soup.find_all(class_="reports__card")
    for article in articles :
        lien = article.find("a", attrs={"href":True})["href"]
        date_cleared = slugify(article.contents[3].contents[1].text)
        date_splited = date_cleared.split('-', 3)
        date_converted = date_splited[2] + "-" + convert_month(date_splited[1]) + "-" + date_splited[0]
        title_cleared = slugify(article.contents[3].contents[3].text)
        file_name = date_converted + "---" + title_cleared
        data = {"filename":file_name, "link":lien}
        all_links.append(data)
    time.sleep(0.1)
    save_file("attempt_3", all_links)

all_links

In [ ]:
# 2) grab article data et save html
from bs4 import BeautifulSoup 
import requests
import re
from unidecode import unidecode
import unicodedata

def grab_data(url) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    return html_soup

def slugify(value, allow_unicode=False):
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

def convert_month(month):
    month_dict = {
        "janvier": 1,
        "février": 2,
        "mars": 3,
        "avril": 4,
        "mai": 5,
        "juin": 6,
        "juillet": 7,
        "août": 8,
        "septembre": 9,
        "octobre": 10,
        "novembre": 11,
        "décembre": 12
    }
    month = unidecode(month.lower())
    for key in month_dict:
        if unidecode(key).startswith(month):
            return str(month_dict[key])
    return None

def work_in_progress(html_soup):
    titre = html_soup.find("div", {'class':'data-author'})
    article = html_soup.find("div", {'class':'data-text'})
    
    data = BeautifulSoup("<html><head><meta charset='utf-8'></head><body></body></html>", "html.parser")
    data.body.append(titre)
    data.body.append(article)
    
    date_cleared = slugify(titre.contents[1].text)
    date_splited = date_cleared.split('-', 3)
    date_converted = date_splited[2] + "-" + convert_month(date_splited[1]) + "-" + date_splited[0]
    title_cleared = slugify(titre.contents[3].text)
    file_name = date_converted + "---" + title_cleared
    return file_name, data

def grab_process_save(url) :
    html_soup = grab_data(url)
    file_name, data = work_in_progress(html_soup)
    with open("html//{}.html".format(file_name), "w", encoding='utf-8') as file:
        file.write(str(data.prettify()))

for i in saucisse :
    url = i["link"]
    grab_process_save(url)
    print("done : ", i["filename"])


In [ ]:
# 3) Loop convert html to pdf
import pdfkit
import os
path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)
directory = 'P:\Code\Prog\IDL_PDF\html'

for html_name in os.listdir(directory):
    if os.path.isfile('pdf\{}.pdf'.format(html_name[:-5])) :
        print("exist")
    else :
        pdfkit.from_file('html\{}'.format(html_name), 'pdf\{}.pdf'.format(html_name[:-5]), configuration=config)
        print("done added : ", html_name)

In [ ]:
# x) Load / Save json
import os.path
import json

def save_file(name, file) :
    with open("{}.json".format(name), "w") as write_file:
        json.dump(file, write_file, indent=4)
    print("Saved")

def load_file(name) :
    if os.path.isfile('{}.json'.format(name)) :
        with open('{}.json'.format(name)) as json_file:
            file = json.load(json_file)
    else :
        print("No previous data detected, initialisation")
        file = {}
        save_file(name, file)
        print("initialisation done")
    return file

saucisse = load_file("all_links")
saucisse
# save_file("save_1", "caca")